# Quickstart Tutorial

In this short introduction to the Masterful AutoML Platform, you will train a model start to finish with Masterful. This example will walk you through creating a simple dataset and model, and using that dataset and model with the Masterful AutoML platform. This example is not intended to create a production ready model, as we will be training on a very small amount of data in order to focus on the key data structures and concepts.


First, import the necessary libraries and register the Masterful package. Registration is necessary to ensure you have a valid authorization key and are licensed to use the Masterful AutoML platform. Make sure to fill in the `<INSERT HERE>` sections with the account ID and authorization key provided to you by Masterful AI.

[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/masterfulai/masterful-docs/blob/main/tutorial_quickstart.ipynb)[![download button](https://www.tensorflow.org/images/download_logo_32px.png)](https://masterful-public.s3.us-west-1.amazonaws.com/933013963/latest/tutorial_quickstart.ipynb) Download this Notebook

In [1]:
import tensorflow as tf
import tensorflow_datasets as tfds

MASTERFUL_ACCOUNT_ID="<INSERT HERE>"
MASTERFUL_AUTHORIZATION_KEY="<INSERT HERE>"
import masterful
masterful = masterful.register(account_id=MASTERFUL_ACCOUNT_ID, 
                               authorization_key=MASTERFUL_AUTHORIZATION_KEY)

MASTERFUL: Your account has been successfully registered. Masterful v0.3.6 is loaded.


In this example, you will use the CIFAR-10 dataset from [Tensorflow Datasets](https://www.tensorflow.org/datasets). You are only going to use 5% of the labeled training data to speed up the overall training time and focus on the end-to-end scenario, rather than particular results.

In [2]:
NUM_CLASSES=10
TRAINING_PERCENTAGE=5
(labeled_dataset, test_dataset) = tfds.load('cifar10', 
                                            as_supervised=True, 
                                            split=[f'train[:{TRAINING_PERCENTAGE}%]', 'test'], 
                                            with_info=False)

Masterful works best with dense labels, so apply one-hot to the sparse labels provided by Tensorflow datasets.

In [3]:
def sparse_to_dense(image, label):
    label = tf.cast(label, tf.int32)
    one_hot_label = tf.one_hot(label, depth=NUM_CLASSES)
    return image, one_hot_label

labeled_dataset = labeled_dataset.map(sparse_to_dense, num_parallel_calls=tf.data.AUTOTUNE)
test_dataset = test_dataset.map(sparse_to_dense, num_parallel_calls=tf.data.AUTOTUNE)

Build a simple model to classify CIFAR-10 data. This is a toy model for demonstration purposes only, and should not be used in a production environment. 

In [4]:
def get_model():
    model = tf.keras.models.Sequential()
    model.add(tf.keras.layers.experimental.preprocessing.Rescaling(1./255,
              input_shape=(32, 32, 3)))
    model.add(tf.keras.layers.Conv2D(16, (3, 3), activation='relu',))
    model.add(tf.keras.layers.MaxPooling2D((2, 2)))
    model.add(tf.keras.layers.Conv2D(32, (3, 3), activation='relu',))
    model.add(tf.keras.layers.MaxPooling2D((2, 2)))
    model.add(tf.keras.layers.Conv2D(64, (3, 3), activation='relu',))
    model.add(tf.keras.layers.MaxPooling2D((2, 2)))
    model.add(tf.keras.layers.GlobalAveragePooling2D())
    model.add(tf.keras.layers.Dense(NUM_CLASSES))
    return model

The first step when using Masterful is to specify some design choices about your model by using `masterful.spec.create_model_and_data_specs`. Masterful tries to infer as much as possible about your model and dataset, but for things which cannot be inferred, Masterful has created `masterful.ModelSpec` and `masterful.DataSpec` data structures to inform the platform about these choices. 

In the example below, we are letting Masterful know that we are performing a classification task (`masterful.spec.Task.CLASSIFICATION`) with 10 labels (`num_classes=NUM_CLASSES`), and that our input image features are in the range [0,255] (`image_range=masterful.spec.ImageRange.ZERO_255`). Furthermore, we are providing dense labels (`sparse=False`) and that our model is outputting logits rather than a softmax classification (`from_logits=True`). 

In [5]:
model = get_model()

model_spec, data_spec = masterful.spec.create_model_and_data_specs(
    model=model, 
    dataset=labeled_dataset, 
    task=masterful.spec.Task.CLASSIFICATION, 
    image_range=masterful.spec.ImageRange.ZERO_255, 
    num_classes=NUM_CLASSES, 
    sparse=False, 
    from_logits=True,
    )

Now, you are ready to train your model using the Masterful AutoML platform. In the next cell, you will see the call to `masterful.autofit`, which is the entry point to the meta-learning engine of the Masterful AutoML platform. Notice there is no need to batch your data (Masterful will find the optimal batch size for you). No need to shuffle your data (Masterful handles this for you). You don't even need to pass in a validation dataset (Masterful finds one for you). You hand Masterful a model and a dataset, and Masterful will figure the rest out for you.

In [6]:
fit_policy, fit_report = masterful.autofit(
  model, 
  model_spec, 
  labeled_dataset, 
  data_spec,
)

MASTERFUL: Auto-fitting model to datasets.
...
MASTERFUL: Training complete in 1.6272061546643575 minutes.


The model you passed into `masterful.autofit` is now trained and updated in place, so you are able to evaluate it just like any other trained Keras model.

In [7]:
model.evaluate(test_dataset.batch(fit_policy.batch_size))

You have now finished training your first model with the Masterful AutoML platform!

## Next Steps

Now that you have finished this basic tutorial, we suggest exploring the rest of the documentation for more advanced examples and use cases. For example, you can learn how to use unlabeled data to further improve the performance of your model. Or you can learn about other vision tasks such as object detection and segmentation. Don't see a topic that you are interested in? Reach out to us directly on email at learn@masterfulai.com or join our Slack [commmunity](https://www.masterfulai.com/community). We are happy to help walk you through whatever challenge you are facing!